In [1]:
!python -m pip install webdriver-manager chromedriver-autoinstaller selenium
!pip install EbookLib
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 985 kB 7.4 MB/s 
     |████████████████████████████████| 139 kB 44.6 MB/s 
     |████████████████████████████████| 358 kB 46.9 MB/s 
     |████████████████████████████████| 58 kB 5.7 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 111 kB 5.0 MB/s 
  Created wheel for EbookLib: filename=EbookLib-0.17.1-py3-none-any.whl size=38184 sha256=42256c33aac81a7726a7c3990edf4cf85ce12c7e40f28996

In [2]:
import requests, uuid, json
from time import sleep
from tqdm import tqdm

# Add your subscription key and endpoint
subscription_key = "1d79a0c2bf56409f8f63f51dc388dbe2"
endpoint = "https://api.cognitive.microsofttranslator.com"

# Add your location, also known as region. The default is global.
# This is required if using a Cognitive Services resource.
location = "eastus"

path = '/translate'
constructed_url = endpoint + path

params = {
    'api-version': '3.0',
    'from': 'en',
    'to': 'vi',
}
constructed_url = endpoint + path

headers = {
    'Ocp-Apim-Subscription-Key': subscription_key,
    'Ocp-Apim-Subscription-Region': location,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
}

In [3]:
size_text = 0
def by_microsoft(translation):
    global size_text
    size_text += len(translation)
    body = [{
        'text': translation, 
    }]

    request = requests.post(constructed_url, params=params, headers=headers, json=body)
    response = request.json()
    response = response[0]['translations'][0]['text']
    translated = []
    eng = translation.split('.')
    vi = response.split('.')
    for i in range(len(eng)):
        translated.append('\n\n\n' + eng[i])
        if i < len(vi):
            translated.append('\n\n' + vi[i])
    return '.'.join(translated)

In [4]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import urllib.request
import shutil

import json
import numpy as np
import pandas as pd
import re
import os
import datetime

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import email, smtplib, ssl

from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

from ebooklib import epub
from datetime import datetime
from google.colab import files


chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument('--window-size=1920,1080')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=chrome_options)

# driver = webdriver.Chrome()
driver.get('https://www.wired.com/')

def xpath_send_key(xpath_, string_):
    driver.find_element(By.XPATH, xpath_).send_keys(string_)

def xpath_click(xpath_):
    driver.find_element(By.XPATH, xpath_).click()

In [5]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
articles = set()
for s in soup.find_all('a', {'data-recirc-pattern': 'summary-item'}):
    if 'https' not in s['href']:
        articles.add(s['href'])
articles

{'/1994/09/moon-land/',
 '/2009/11/ff-vanish2/',
 '/2014/08/the-most-fascinating-profile-youll-ever-read-about-a-guy-and-his-boring-startup/',
 '/gallery/best-college-and-dorm-essentials-gear/',
 '/gallery/best-monitors/',
 '/gallery/best-office-chairs/',
 '/gallery/best-travel-adapters',
 '/gallery/the-best-backpacks-for-work/',
 '/review/google-pixel-buds-pro/',
 '/review/insta360-one-rs-1-inch-360-edition/',
 '/story/advertising-services-devices-rant/',
 '/story/apple-ios-vpn-data-leak',
 '/story/artists-rage-against-machines-that-mimic-their-work/',
 '/story/as-wildfires-get-more-extreme-observatories-are-at-greater-risk/',
 '/story/back-to-school-deals-2022/',
 '/story/best-rapid-at-home-covid-19-test-kits/',
 '/story/best-student-discounts/',
 '/story/big-business-burying-carbon-dioxide-capture-storage/',
 '/story/drought-causing-floods/',
 '/story/game-of-thrones-vs-lord-of-the-rings-streaming-wars',
 '/story/how-inflation-reduction-act-climate-bill-save-you-money/',
 '/story/in

In [6]:
def scrap_article():
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    title = soup.find('main').find('h1').text

    forbidden_char = ['<', '>', ':', '"', "'", '/', '\\', '|', '?', '*']

    file_name = title + '.txt'
    for c in forbidden_char:
        file_name = file_name.replace(c, '_')

    if file_name in os.listdir('articles/'):
        return None

    need_translatation = ""
    for s in soup.find_all('div', {'class':'body__inner-container'}):
        for paragraph in s.find_all('p'):
            need_translatation += paragraph.text
    try:
        translated = by_microsoft(need_translatation)
    except Exception as e:
        print(e)
        return None
    
    f = open('articles/' + file_name, 'w', encoding='utf-8')
    f.write('# ' + '<h1>' + title + '</h1>'  + '\n')
    f.write(translated)
    f.close()
    
    return (title, translated)

In [7]:
def send_to_kindle(sender_email, password, receiver_email, filename):


    subject = "Test"
    body = "This is an email with attachment sent from Python"

    # Create a multipart message and set headers
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = subject
    message["Bcc"] = receiver_email  # Recommended for mass emails

    # Add body to email
    message.attach(MIMEText(body, "plain"))

    # Open PDF file in binary mode
    with open(filename, "rb") as attachment:
        # Add file as application/octet-stream
        # Email client can usually download this automatically as attachment
        part = MIMEBase("application", "octet-stream")
        part.set_payload(attachment.read())

    # Encode file in ASCII characters to send by email    
    encoders.encode_base64(part)

    # Add header as key/value pair to attachment part
    part.add_header(
        "Content-Disposition",
        f"attachment; filename= {filename}",
    )

    # Add attachment to message and convert message to string
    message.attach(part)
    text = message.as_string()

    # Log in to server using secure context and send email
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, text)


In [8]:
NOW = datetime.now().strftime(
    "%d_%m%Y___%H_%M_%S"
)
def create_ebook():
    book = epub.EpubBook()

    # set metadata
    book.set_title(NOW)
    book.set_language('en')

    return book

def add_chapter(book, title, content, chapter_num):

    chapter_num = 'chap_0' + str(chapter_num) +'.xhtml'
    # create chapter
    chapter = epub.EpubHtml(title=title, file_name=chapter_num)
    chapter.content=u'<h1>' + title + u'</h1>' + u'<p>' + content + u'</p>'

    # add chapter
    book.add_item(chapter)

    return book, chapter


def generate_toc(book, chapters):
    # define Table Of Contents
    book.toc = (chapters)

    # add default NCX and Nav file
    book.add_item(epub.EpubNcx())
    book.add_item(epub.EpubNav())

    # define CSS style
    style = 'BODY {color: white;}'
    nav_css = epub.EpubItem(uid="style_nav", file_name="style/nav.css", media_type="text/css", content=style)

    # add CSS file
    book.add_item(nav_css)

    print(type(chapters))
    # basic spine
    book.spine = ['nav', *chapters]
    
    return book


def export_ebook(book, fname):
    # write to the file
    epub.write_epub(fname, book, {})


In [9]:
based_url = 'https://www.wired.com'

os.makedirs('articles', exist_ok=True)

book = create_ebook()
chapters = []
i = 1
for article in tqdm(articles):
    print(article)
    driver.get(based_url + article)
    if 'I Accept' in str(BeautifulSoup(driver.page_source, 'html.parser')):
        xpath_click('//*[@id="onetrust-accept-btn-handler"]')
    scraped = scrap_article()
    if scraped:
        title, translated = scraped
        book, chapter = add_chapter(book, title, translated, i)
        i += 1

        chapters.append(chapter)

book = generate_toc(book, tuple(chapters))
fout =  NOW + '.epub'
export_ebook(book, fout)


  0%|          | 0/33 [00:00<?, ?it/s]

/review/google-pixel-buds-pro/


  3%|▎         | 1/33 [00:08<04:29,  8.42s/it]

/gallery/best-travel-adapters


  6%|▌         | 2/33 [00:48<14:06, 27.29s/it]

...

0
/gallery/best-office-chairs/


 97%|█████████▋| 32/33 [15:45<00:34, 34.01s/it]

0
/story/as-wildfires-get-more-extreme-observatories-are-at-greater-risk/


100%|██████████| 33/33 [16:13<00:00, 29.49s/it]

0
<class 'tuple'>


In [10]:
files.download(fout)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
send_to_kindle('quocdat25@gmail.com', '#####', 'quocdat25_new@kindle.com', fout)

OBS